In [2]:

# ra, dec, rot = header["RA_DTU"], header["DEC_DTU"], header["ROLL_DTU"]
ra, dec, rot = 100, 50, 50


def quat_multiply(self, quaternion0, quaternion1):
    x0, y0, z0, w0 = np.split(quaternion0, 4, 1)
    x1, y1, z1, w1 = np.split(quaternion1, 4, 1)

    result = np.array(
        (
            x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
            -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
            x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0,
            -x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
        ),
        dtype=np.float64,
    )
    return np.transpose(np.squeeze(result))


#%%
def prod_quat(q=None, p=None):
    # Cette fonction calcule le produit de deux quaternions
    print(q, p)
    # quat = np.array([[q(1),- q(2),- q(3),- q(4)],[q(2),q(1),- q(4),q(3)],[q(3),q(4),q(1),- q(2)],[q(4),- q(3),q(2),q(1)]]) * p
    q2 = np.array(
        [
            [q[0], -q[1], -q[2], -q[3]],
            [q[1], q[0], -q[3], q[2]],
            [q[2], q[3], q[0], -q[1]],
            [q[3], -q[2], q[1], q[0]],
        ]
    )
    # quat = np.dot(q2, p)
    print(q2, p)
    quat = np.matmul(q2, p.T)
    return quat


def quat_to_mat(q):
    q0 = q[0]
    q1 = q[1]
    q2 = q[2]
    q3 = q[3]
    M = np.array(
        [
            [
                2 * (q0 ** 2 + q1 ** 2) - 1,
                2 * (q1 * q2 + q0 * q3),
                2 * (q1 * q3 - q0 * q2),
            ],
            [
                2 * (q1 * q2 - q0 * q3),
                2 * (q0 ** 2 + q2 ** 2) - 1,
                2 * (q2 * q3 + q0 * q1),
            ],
            [
                2 * (q1 * q3 + q0 * q2),
                2 * (q2 * q3 - q0 * q1),
                2 * (q0 ** 2 + q3 ** 2) - 1,
            ],
        ]
    )
    return M


def alpha_delta_rot(Q=None):
    # Hypoth se:
    # On suppose que l'axe de vis e sortant du SST a pour coordonn es [0 0 1]

    # Entr e
    # Q: quaternion entre le rep re inertiel et le rep re CU

    # Sorties:
    # alpha: ascension droite, entre 0 et 2*pi (rad)
    # delta: d clinaison, entre -pi/2 et +pi/2 (rad)
    # rot: rotation de champ, au sens du rep re inertiel, entre -pi et +pi (rad)

    M = quat_to_mat(Q)
    epsilon = 1e-06

    print(M.shape, M[2, 2], M[2, 2].shape)
    if np.abs(M[2, 2]) < 1 - epsilon:
        delta = np.arcsin(M[2, 2])
        alpha = np.arctan2(M[1, 2], M[0, 2])
        rot = -np.arctan2(M[2, 1], M[2, 0])
    else:
        if M[2, 2] > 1 - epsilon:
            delta = np.pi / 2
            alpha = np.arctan2(M[1, 0], M[0, 0])
            rot = 0
        else:
            delta = -np.pi / 2
            alpha = np.arctan2(M[1, 0], M[0, 0])
            rot = 0

    if alpha < 0:
        alpha = alpha + 2 * np.pi

    return alpha, delta, rot


def alpha_delta_rot_to_quat_CU(alpha=None, delta=None, rot=None):
    # Entrées
    # alpha : ascension droite (rad)
    # delta : declinaison (rad)
    # rot : rotation de champ (rad)

    # Sorties
    # q_I_CU: quaternion de passage du repère inertiel I au repère CU avec
    # l'axe de visée Z pointant les coordonnées équatoriales (alpha,delta) et
    # une rotation de champ finale (rot) autour de Z_CU
    asc = alpha
    dec = delta
    q1 = np.transpose(np.array([[np.cos(asc / 2.0)], [0], [0], [np.sin(asc / 2.0)]]))
    q2 = np.transpose(
        np.array(
            [
                [np.cos((np.pi / 2.0 - dec) / 2.0)],
                [0],
                [np.sin((np.pi / 2.0 - dec) / 2.0)],
                [0],
            ]
        )
    )
    q_I_CU_0 = prod_quat(q1[0], q2[0])
    q3 = np.transpose(np.array([[np.cos(rot / 2.0)], [0], [0], [np.sin(rot / 2.0)]]))
    q_I_CU = prod_quat(q_I_CU_0, q3)
    return q_I_CU


#%%


#%%

##
# quaternion DTU à 6h04h04 (image guider à 6h04:47 mais 50s d'avance pour
# le guider)
# mettre ici directement les biais envoyés par TC depuis le CC CNES
import numpy as np

biais_el = -615 / 3600 * np.pi / 180
biais_ce = 5330 / 3600 * np.pi / 180
biais_el = 100 / 3600 * np.pi / 180
biais_ce = 0 / 3600 * np.pi / 180
# biais_el =1.828 * np.pi / 180
# biais_ce = 0.036 * np.pi / 180
alpha_DTU = ra * np.pi / 180
delta_DTU = dec * np.pi / 180
psi_DTU = rot * np.pi / 180
dtu_quat = alpha_delta_rot_to_quat_CU(alpha_DTU, delta_DTU, psi_DTU).T[0] #[::-1]
# dtu_quat = np.array([0.865754, 0.459764, 0.175396, 0.091233])
# dtu_quat= [1 0 0 0];
alpha, delta, psi = alpha_delta_rot(dtu_quat)
# alpha, psi = psi, alpha
print(alpha_DTU, "=", alpha)
print(delta_DTU, "=", delta)
print(psi_DTU, "=", psi)

dq = np.array([1, biais_ce / 2, biais_el / 2, 0])
# dq = np.zeros((4))
dq = dq / np.linalg.norm(dq)
dtu_quat_corrige = prod_quat(dtu_quat, dq)
alpha_corr, delta_corr, psi_corr = alpha_delta_rot(dtu_quat_corrige)

print(
    "Biais DTU appliqués : el=%2.8f arcsec, ce=%2.8f arcsec\n"
    % (biais_el * 180 / np.pi * 3600, biais_ce * 180 / np.pi * 3600)
)
print(
    "Q_DTU brut : alpha = %2.8f deg, delta = %2.8f deg, psi = %2.8f deg\n"
    % (alpha * 180 / np.pi, delta * 180 / np.pi, psi * 180 / np.pi)
)
print(
    "Q_DTU corrigé : alpha = %2.8f deg, delta = %2.8f deg, psi = %2.8f deg\n"
    % (alpha_corr * 180 / np.pi, delta_corr * 180 / np.pi, psi_corr * 180 / np.pi)
)



[0.64278761 0.         0.         0.76604444] [0.93969262 0.         0.34202014 0.        ]
[[ 0.64278761 -0.         -0.         -0.76604444]
 [ 0.          0.64278761 -0.76604444  0.        ]
 [ 0.          0.76604444  0.64278761 -0.        ]
 [ 0.76604444 -0.          0.          0.64278761]] [0.93969262 0.         0.34202014 0.        ]
[ 0.60402277 -0.26200263  0.21984631  0.71984631] [[0.90630779 0.         0.         0.42261826]]
[[ 0.60402277  0.26200263 -0.21984631 -0.71984631]
 [-0.26200263  0.60402277 -0.71984631  0.21984631]
 [ 0.21984631  0.71984631  0.60402277  0.26200263]
 [ 0.71984631 -0.21984631 -0.26200263  0.60402277]] [[0.90630779 0.         0.         0.42261826]]
(3, 3) 0.7660444431189779 ()
1.7453292519943295 = 2.2689280275926285
0.8726646259971648 = 0.8726646259971645
0.8726646259971648 = -1.7453292519943293
[ 0.24321035 -0.14454396  0.30997552  0.90767337] [9.99999971e-01 0.00000000e+00 2.42406833e-04 0.00000000e+00]
[[ 0.24321035  0.14454396 -0.30997552 -0.907